# Seleccion de variables 

Observamos con anterioridad que la mayoria de las variables categoricas tienen alta cardinialidad.En este notebook queremos trabajar las variables categoricas con ingeneria de variables con el objetivo de sumarlas al dataframe obtenido en el analisis de PCA. 

## Import libraries

In [1]:
# Sirve para actualizar el modulo despues de haber cambiado algo en el codigo de los modulos importados
%load_ext autoreload
%autoreload 2

In [2]:
#Importamos las librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter
from seaborn import violinplot
%matplotlib inline
# Modulo para trabajar con los directorios del proyecto
from src.utils.paths import data_dir

In [3]:
# Modulos propios del proyecto que se encuentran en el directorio src
from src.utils.paths import data_dir
from src.visualization.word_cloud import words_cloud
from src.utils.paths import references_dir

## Set up

In [4]:
# Configuracion de opciones

# Configuramos para que muestre siempre todas las columnas disponibles
pd.options.display.max_columns = None

# Cambio del tamaño de los plots para seaborn
sns.set(rc={'figure.figsize':(10,5)})

# Definimos el color para los plots
COLOR_VINO_TINTO = "#7C3030"
COLOR__VINO_BLANCO = "#ffffbf"

## Read Data

In [5]:
# Obtenemos la ruta de los csv's que vamos a utilizar
WINES_MERGE_CSV_PATH = data_dir("intermediate", "df_wine_merged.csv")
WINES_PCA_CSV_PATH = data_dir("intermediate", "df_wine_pca.csv")

print(f"{WINES_MERGE_CSV_PATH}")
print(f"{WINES_PCA_CSV_PATH}")

c:\Users\cvigo\data-projects\proyecto-final-vinos\vinos-corderhouse\data\intermediate\df_wine_merged.csv
c:\Users\cvigo\data-projects\proyecto-final-vinos\vinos-corderhouse\data\intermediate\df_wine_pca.csv


In [6]:
# Leemos los csv's y los guardamos en un dataframe
df_wine_completo = pd.read_csv(WINES_MERGE_CSV_PATH)
df_wine_pca = pd.read_csv(WINES_PCA_CSV_PATH)

print(f"{df_wine_completo.shape=}")
print(f"{df_wine_pca.shape=}")

df_wine_completo.shape=(5013, 21)
df_wine_pca.shape=(5013, 6)


# Analisis de cardinalidad de las variables categoricas

In [7]:
df_wine_completo.dtypes

winery                   object
wine                     object
year                      int64
rating                  float64
region                   object
price                   float64
type                     object
body                    float64
fixed_acidity           float64
volatile_acidity        float64
citric_acid             float64
residual_sugar          float64
chlorides               float64
free_sulfur_dioxide     float64
total_sulfur_dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
style                    object
dtype: object

In [8]:
# filtrar los datos de tipo object
df_wine_completo_object = df_wine_completo.select_dtypes(include=['object'])

df_wine_completo_object.head()

,winery,wine,region,type,style
0,Vinedos de Paganos,La Nieta,Rioja,Rioja Red,red
1,Emilio Moro,Malleolus de Valderramiro,Ribera del Duero,Ribera Del Duero Red,red
2,Emilio Moro,Malleolus de Sanchomartin,Ribera del Duero,Ribera Del Duero Red,red
3,Bodegas Mauro,Terreus Paraje de Cueva Baja,Castilla y Leon,Tempranillo,red
4,Vinedos de Paganos,La Nieta,Rioja,Rioja Red,red


In [9]:
cardinalidad = pd.DataFrame({'Cantidad' : df_wine_completo_object.nunique(), 'Porcentaje' : (df_wine_completo_object.nunique()) / (df_wine_completo_object.shape[0]) * (100)})
cardinalidad

,Cantidad,Porcentaje
winery,80,1.595851
wine,204,4.069420
region,22,0.438859
type,15,0.299222
style,2,0.039896


Las variables categoricas `winery`, `wine`, `region` y `type` tiene alta cardinalidad por lo que no es conveniente sumarlas a nuestro algoritmo. Procedemos a trabajar la variable `style`

## Ingenieria de variables categoricas

### Style

In [10]:
## Nos quedamos con la variable style 
df_style = df_wine_completo_object[['style']]

df_style.head()

,style
0,red
1,red
2,red
3,red
4,red


In [11]:
# Transfomamos la variable categorica style en una variable nominal
df_style = df_style.replace({'red':'2','white':'1'})
df_style.head()

,style
0,2
1,2
2,2
3,2
4,2


### Quality

In [12]:
#Comenzamos con la preparacion de nuestros datos
df_pca_dummy = pd.get_dummies(df_wine_pca, drop_first=True)
df_pca_dummy.head()

,PC1,PC2,PC5,PC6,PC7,quality_low
0,-0.305926,0.757400,-0.325291,-0.214702,0.190483,1
1,-0.084796,0.690995,0.196268,0.329449,0.050096,1
2,-0.301476,0.894188,-0.208230,-0.216697,0.136748,1
3,-0.076054,0.651771,0.200872,0.325854,0.047974,1
4,-0.088164,0.516868,0.584555,0.085229,0.087850,0


## Merge de los variables que utilizaremos para trabajar los algoritmos

In [13]:
df_merged = pd.merge(df_pca_dummy, df_style, left_index=True, right_index=True)
print(f"El shape de df_merged es: {df_merged.shape}")
df_merged.sample(5)

El shape de df_merged es: (5013, 7)


,PC1,PC2,PC5,PC6,PC7,quality_low,style
4612,-0.121694,-0.011782,0.042981,-0.130631,0.074071,1,1
1055,0.319687,-0.114384,0.096003,0.015267,-0.006333,0,1
1509,0.316165,-0.042201,-0.097317,0.001171,-0.135371,0,1
834,-0.483138,-0.062793,-0.312197,-0.052575,0.172142,1,1
1538,0.146683,-0.095343,0.012807,-0.137203,0.196400,0,1


In [14]:
# Por ultimo guardamos el dataframe mergeado en un archivo csv
WINE_FINAL_CSV_PATH = data_dir("processed", "df_wine_final.csv")
df_merged.to_csv(WINE_FINAL_CSV_PATH, index=False)
